# Merge Covariate
This is the module where the output of factor analysis were merged into 1 covariate file that can be fed into both APEX and tensorQTL

### Input
1. factor+cov file as output from peer or BiCV factor module, It is assumed it to have columns as #id + samplesname and each rows is a covariateor factor (start with factor_)

1. pca file as output from the PCA module

### Output
1. PCA + Factor + Covariate file


In [2]:
[global]
import os
# Work directory & output directory
parameter: wd = path("./")
# The filename name for output data
parameter: container = 'gaow/twas'
# name for the analysis output
parameter: name = 'ROSMAP'
# For cluster jobs, number commands to run per job
parameter: job_size = 1
# Wall clock time expected
parameter: walltime = "5h"
# Memory expected
parameter: mem = "16G"
# Number of threads
parameter: numThreads = 20
# Input 1
parameter: factor_and_covariate = path
# Input 2
parameter: PC = path

In [ ]:
[pca_factor_merge]
## PCA models
input: PC, factor_and_covariate
output:  f'{wd:a}/{_input[1]:bn}.pca.cov'
R: expand= "$[ ]", stderr = f'{_output:n}.stderr', stdout = f'{_output:n}.stdout'
        library("dplyr")
        library("tibble")
        library("readr")
        library("modelr")
        library("purrr")
        pca_output = readRDS("$[_input[0]]")$pc_scores
        mtx = pca_output%>%select(contains("PC"))%>%t()
        colnames(mtx) <- pca_output$IID
        mtx = mtx%>%as_tibble()%>%mutate("#id" = rownames(mtx))%>%select("#id",everything())
        factor_cov = read_delim("$[_input[1]]","\t")
        output = bind_rows(factor_cov,mtx)
        output%>%write_delim("$[_output]","\t")